# Plot structure and PCA components

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Load some information about meshes.

In [ ]:
with np.load('../data/vertices_stats.npz') as data:
    geom = dict()
    geom['mean'] = data['mean']
    geom['std'] = data['std']
    # Structure per face
    geom['mean_face'] = data['mean_face']
    geom['tri'] = np.load('../data/tris.npy') - 1

In [ ]:
def plot_face(v, tri, overlay=None, multiple_v=False, multiple_o=False):

    if overlay is None:
        color = 'darkgray'
        minn, maxx = None, None
    else:
        color = None
        maxx = overlay.max()
        minn = -maxx
            
    layout = go.Layout(
        scene=dict(xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)),
        scene_camera=dict(up=dict(x=0, y=0, z=1), center=dict(x=0, y=0, z=0), eye=dict(x=1.5, y=0, z=0))
    )
    
    if multiple_v or multiple_o:
        
        n_plots = v.shape[-1] if multiple_v else overlay.shape[-1]
        n_rows = int(np.ceil(n_plots / 5))
        n_cols = min(5, n_plots)
        specs = [[{'type': 'surface'}] * n_cols] * n_rows
        fig = make_subplots(rows=n_rows, cols=n_cols,
                            specs=specs,
                            shared_xaxes=True, shared_yaxes=True)
        #layout['scene_camera']['eye']['x'] = 1.5 + 0.75 * n_cols
        for i in range(n_plots):
            if multiple_o:
                this_o = overlay[:, i]
            else:
                this_o = overlay

            if multiple_v:
                this_v = v[:, :, i]
            else:
                this_v = v

            mesh = go.Mesh3d(
                y=this_v[:, 0], z=this_v[:, 1], x=this_v[:, 2],
                j=tri[:, 0], k=tri[:, 1], i=tri[:, 2],
                intensity=this_o, colorscale='RdBu',
                cmax=maxx, cmin=minn, reversescale=True,
                color=color
            )
            row_idx = i // n_cols + 1
            col_idx = i % n_cols + 1
            fig.add_trace(mesh, row=row_idx, col=col_idx)        
    else:
        mesh = go.Mesh3d(
            y=v[:, 0], z=v[:, 1], x=v[:, 2],
            j=tri[:, 0], k=tri[:, 1], i=tri[:, 2],
            intensity=overlay, colorscale='RdBu',
            cmax=maxx, cmin=minn, reversescale=True,
            color=color
        )
        fig = go.Figure(data=[mesh], layout=layout)

    fig.update_scenes(layout['scene'])
    
    return fig

How does the average face look like? We'll plot the average face with the standard deviation (across X, Y, Z directions summed) on top:

In [ ]:
fig = plot_face(geom['mean'], geom['tri'], overlay=geom['std'].sum(axis=1))
fig.show()

Per face:

In [ ]:
fig = plot_face(geom['mean_face'][:, :, :3], geom['tri'], multiple_v=True)
fig.show()

In [ ]:
with np.load('../models/featurespace-vertexmotionPCA_norm-n_weights.npz') as data:
    mu, w = data['mu'], data['w']
fig = plot_face(geom['mean'], geom['tri'], overlay=mu)
fig.show()

In [ ]:
with np.load('../models/featurespace-vertexmotionPCA_norm-n_weights.npz') as data:
    mu, w = data['mu'], data['w']
fig = plot_face(geom['mean'], geom['tri'], overlay=w.T[:, :5], multiple_o=True)
fig.show()

In [ ]:
with np.load('../models/featurespace-vertexmotionPCA_norm-y_weights.npz') as data:
    mu, w = data['mu'], data['w']
fig = plot_face(geom['mean'], geom['tri'], overlay=w.T[:, :5], multiple_o=True)
fig.show()

In [ ]:
with np.load('../models/featurespace-vertexPCA_frame-01_norm-n_weights.npz') as data:
    mu, w = data['mu'], data['w']
    w = w.reshape((50, 31049, 3))

fig = plot_face(geom['mean'], geom['tri'], overlay=w[:5, :, 0].T, multiple_o=True)
fig.show()

In [ ]:
with np.load('../models/featurespace-vertexPCA_frame-15_norm-n_weights.npz') as data:
    mu, w = data['mu'], data['w']
    w = w.reshape((50, 31049, 3))
    w = np.moveaxis(w, 0, 2)

norm = np.sqrt((w ** 2).sum(axis=1))
fig = plot_face(mu.reshape((31049, 3, 1)) + w[:, :, :5] * 1000, geom['tri'], overlay=norm,
                multiple_v=True, multiple_o=True)
fig.show()

In [ ]:
with np.load('../models/featurespace-vertexPCA_frame-15_norm-y_weights.npz') as data:
    mu, w = data['mu'], data['w']
    w = w.reshape((50, 31049, 3))
    w = np.moveaxis(w, 0, 2)

norm = np.sqrt((w ** 2).sum(axis=1))
fig = plot_face(mu.reshape((31049, 3, 1)) + w[:, :, :5] / geom['std'][..., None], geom['tri'], overlay=norm,
                multiple_v=True, multiple_o=True)
fig.show()